<a href="https://colab.research.google.com/github/kailen-mccauley/Covid-19_CNN/blob/main/Covid_model_using_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision pandas neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.5/481.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67673 sha256=d4ab65486434c932c78c6c43779768b247033bc967981900b8b50eb0a2a27c43
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44ec4c3

In [ ]:
from PIL import Image
import pandas as pd

# Set the paths to your image folder and label file
image_folder_path = '/content/drive/MyDrive/VIP /COVID_DATA/Emory COVID All Data'
label_file_path = '/content/drive/MyDrive/VIP /COVID_DATA/Rad Labels1.xlsx'

# Read the labels from the Excel file
labels_df = pd.read_excel(label_file_path)

In [ ]:
import neptune

batchSize = 64
numEpochs = 10
dropout = 0.5
learningRate = 0.0001
momentum = 0.9
weightDecay = 0.001

run = neptune.init_run(
    project="kailen.mccauley/CovidX-RayNetwork",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiNzM4NWQ4Zi05NzY1LTQ2NjktYjRmYi1mNTUwM2QyMDRlMjYifQ==",
)

params = {
    "Model": "VGG16",
    "Batch Size": batchSize,
    "Epochs": numEpochs,
    "Dropout": dropout,
    "Optimizer": "Adam",
    "Learning Rate": learningRate,
    "Momentum": momentum,
    "Weight Decay": weightDecay,
    "Transform" :"Normalize, Horizontal Flip, and Gray Scale",
    }
run["parameters"] = params

<ipython-input-3-19cb1cb75b9c>:10: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/kailen.mccauley/CovidX-RayNetwork/e/COV-34


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os

if(torch.cuda.is_available()):
  device = torch.device("cuda")
  print("CUDA is working!")
else:
  device = torch.device("cpu")
  print("CUDA is NOT working!")


class CustomDataset(Dataset):
    def __init__(self, image_folder, labels_df, transform=None):
        self.image_folder = image_folder
        self.labels_df = labels_df
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.labels_df.iloc[idx, 0])
        image = Image.open(img_name+'.jpg')
        label = self.labels_df.iloc[idx, 9]  # Col. 9 is where the mean doctor label is

        if self.transform:
            image = self.transform(image)

        return image, label

CUDA is working!


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)),
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),
])


dataset = CustomDataset(image_folder=image_folder_path, labels_df=labels_df, transform=transform)
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)

In [ ]:
from torch.utils.data import random_split


train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size

# Splits whole dataset into train and test without overlap
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)


classes = ('0','1','2','3')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Define our pre-trained model
vgg16 = models.vgg16(pretrained=True)
vgg16 = vgg16.to(device)

# Modify the classifier since we only have 4 classes
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 4)  # Change 10 to 4 since vgg normally had 10 but
vgg16 = vgg16.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=learningRate, momentum=momentum, weight_decay = weightDecay)


# Training script
for epoch in range(numEpochs):
    running_train_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        images = images.repeat(1, 3, 1, 1)
        optimizer.zero_grad()

        outputs = vgg16(images)
        trainLoss = criterion(outputs, labels)
        trainLoss.backward()
        optimizer.step()

        running_train_loss += trainLoss.item()
        outputs = outputs.detach()
        trainLoss = trainLoss.detach()

    run["Train/Epoch/Loss"].append(running_train_loss / train_size)
    print(f"Epoch {epoch + 1}, Loss: {running_train_loss / len(train_loader)}")


    # Testing
    correct = 0
    total = 0
    class_correct = [0 for _ in range(4)]
    class_total = [0 for _ in range(4)]
    running_test_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            images = images.repeat(1, 3, 1, 1)
            outputs = vgg16(images)


            testLoss = criterion(outputs, labels)
            running_test_loss += testLoss
            running_test_loss = running_test_loss.detach()


            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    run["Test/Validation Loss"].append(running_test_loss  / test_size)
    run["Test/Overall Accuracy"].append(100 * correct / total)

    # Print overall accuracy
    print("TEST RESULTS FROM EPOCH: "+str(epoch))
    print(f"Accuracy on the test set: {100 * correct / total}%")

    # Print accuracy for each category
    for i in range(4):  # Change 10 to 4
        print(f"Accuracy for class {i}: {100 * class_correct[i] / class_total[i]}%")
        run["Test/Class "+str(i)+" Accuracy"].append(100 * class_correct[i] / class_total[i])


KeyboardInterrupt: ignored